# Method 2: Oracle Extraction - Llama Batch Processing

**Oracle Strategy**: Use ground truth document type to select correct extraction schema.

**Key differences from baseline:**
- No classification step - ground truth provides document type
- Always uses correct field set (perfect classification)
- 5 fields for bank statements, 14 fields for invoice/receipt

**Purpose**: Quantify classification penalty by eliminating classification errors.

In [ ]:
#Cell 1
# Imports and setup
%load_ext autoreload
%autoreload 2

import os
os.environ['EVALUATION_METHOD'] = 'order_aware_f1'

import warnings
import yaml
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from PIL import Image as PILImage
from rich import print as rprint
from rich.console import Console

from common.evaluation_metrics import (
    load_ground_truth,
    calculate_field_accuracy_with_method
)
from common.extraction_parser import (
    discover_images,
    parse_extraction_response
)
from common.gpu_optimization import emergency_cleanup
from common.llama_model_loader_robust import load_llama_model_robust

console = Console()
warnings.filterwarnings('ignore')

rprint("[green]✅ Imports loaded successfully[/green]")

In [ ]:
#Cell 2
# Pre-emptive memory cleanup
rprint("[bold red]🧹 PRE-EMPTIVE V100 MEMORY CLEANUP[/bold red]")
rprint("[yellow]Clearing any existing model caches before loading...[/yellow]")

emergency_cleanup(verbose=True)

rprint("[green]✅ Memory cleanup complete - ready for model loading[/green]")

In [ ]:
#Cell 3
# Environment-specific base paths
ENVIRONMENT_BASES = {
    'sandbox': '/home/jovyan/nfs_share/tod',
    'efs': '/efs/shared/PoC_data'
}
base_data_path = ENVIRONMENT_BASES['efs']

CONFIG = {
    # Model settings
    # 'MODEL_PATH': "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct",
    'MODEL_PATH': "/efs/shared/PTM/Llama-3.2-11B-Vision-Instruct",
    
    # Batch settings
    # 'DATA_DIR': f'{base_data_path}/evaluation_data',
    # 'GROUND_TRUTH': f'{base_data_path}/evaluation_data/ground_truth.csv',
    'DATA_DIR': "/home/jovyan/_LMM_POC/evaluation_data/bank",
    'GROUND_TRUTH': "/home/jovyan/_LMM_POC/evaluation_data/bank/bank_gt.csv",
    
    # 'DATA_DIR': "/home/jovyan/shared_PoC_data/annotation_images_edited_short_filename",
    # 'GROUND_TRUTH': "/home/jovyan/shared_PoC_data/evaluation_data/ground_truth_2025_11_03/ground_truth_2025_11_03_with_tods_bank_images.csv",
    
    'OUTPUT_BASE': f'{base_data_path}/LMM_POC/output',
    'MAX_IMAGES': None,
    
    # Verbosity
    'VERBOSE': True,
    'SHOW_PROMPTS': True,
    
    # Model settings
    'USE_QUANTIZATION': False,
    'DEVICE_MAP': 'auto',
    'MAX_NEW_TOKENS': 2000,
    'TORCH_DTYPE': 'bfloat16',
    'LOW_CPU_MEM_USAGE': True,
    
    # Preprocessing
    'ENABLE_PREPROCESSING': True,
    'PREPROCESSING_MODE': 'adaptive',
    'SAVE_PREPROCESSED': False,
    'PREPROCESSED_DIR': None,
}

rprint("[green]✅ Configuration set[/green]")
rprint(f"[cyan]📂 Data: {CONFIG['DATA_DIR']}[/cyan]")
rprint(f"[cyan]📊 Ground truth: {CONFIG['GROUND_TRUTH']}[/cyan]")
rprint(f"[cyan]🤖 Model: {CONFIG['MODEL_PATH']}[/cyan]")

In [ ]:
#Cell 4
# Setup output directories
OUTPUT_BASE = Path(CONFIG['OUTPUT_BASE'])
if not OUTPUT_BASE.is_absolute():
    OUTPUT_BASE = Path.cwd() / OUTPUT_BASE

BATCH_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

OUTPUT_DIRS = {
    'base': OUTPUT_BASE,
    'batch': OUTPUT_BASE / 'batch_results',
    'csv': OUTPUT_BASE / 'csv',
    'visualizations': OUTPUT_BASE / 'visualizations',
    'reports': OUTPUT_BASE / 'reports'
}

for dir_path in OUTPUT_DIRS.values():
    dir_path.mkdir(parents=True, exist_ok=True)

rprint("[green]✅ Output directories created[/green]")

In [ ]:
#Cell 5
# Load model
rprint("[bold green]Loading Llama model...[/bold green]")

model, processor = load_llama_model_robust(
    model_path=CONFIG['MODEL_PATH'],
    use_quantization=CONFIG['USE_QUANTIZATION'],
    device_map=CONFIG['DEVICE_MAP'],
    max_new_tokens=CONFIG['MAX_NEW_TOKENS'],
    torch_dtype=CONFIG['TORCH_DTYPE'],
    low_cpu_mem_usage=CONFIG['LOW_CPU_MEM_USAGE'],
    verbose=CONFIG['VERBOSE']
)

rprint("[bold green]✅ Model loaded successfully[/bold green]")

In [ ]:
#Cell 6
# Load oracle extraction prompts from YAML files
# We need separate prompts for invoice/receipt (14 fields) and bank statement (5 fields)

PROMPT_FILES = {
    'invoice': 'prompts/generated/llama_invoice_prompt.yaml',
    'receipt': 'prompts/generated/llama_receipt_prompt.yaml',
    'bank_statement': 'prompts/generated/llama_bank_statement_prompt.yaml'
}

ORACLE_PROMPTS = {}

for doc_type, prompt_file in PROMPT_FILES.items():
    prompt_path = Path(prompt_file)
    
    if not prompt_path.exists():
        rprint(f"[red]❌ Prompt file not found: {prompt_file}[/red]")
        continue
    
    with open(prompt_path, 'r') as f:
        prompt_data = yaml.safe_load(f)
    
    # Extract the prompt from the YAML structure
    # Structure: prompts -> {doc_type} -> prompt
    if 'prompts' in prompt_data and doc_type in prompt_data['prompts']:
        ORACLE_PROMPTS[doc_type] = prompt_data['prompts'][doc_type]['prompt']
        rprint(f"[green]✅ Loaded {doc_type} prompt from {prompt_file}[/green]")
    else:
        rprint(f"[yellow]⚠️  Could not find prompt in expected structure for {doc_type}[/yellow]")

rprint(f"[cyan]📋 Loaded {len(ORACLE_PROMPTS)} oracle prompts[/cyan]")

In [ ]:
#Cell 6.5
# Display oracle prompts
if CONFIG.get('SHOW_PROMPTS', True) and ORACLE_PROMPTS:
    console.rule("[bold cyan]Oracle Extraction Prompts[/bold cyan]")
    
    for doc_type, prompt in ORACLE_PROMPTS.items():
        console.rule(f"[cyan]{doc_type.upper()}[/cyan]")
        # Show first 500 chars of each prompt
        preview = prompt[:500] + "..." if len(prompt) > 500 else prompt
        print(preview)
    
    console.rule("[bold cyan]End of Prompts[/bold cyan]")
else:
    rprint("[dim]Prompt display disabled[/dim]")

In [ ]:
#Cell 7
# Discover images and load ground truth
data_dir = Path(CONFIG['DATA_DIR'])
if not data_dir.is_absolute():
    data_dir = Path.cwd() / data_dir

all_images = discover_images(str(data_dir))

# Apply preprocessing if enabled
if CONFIG['ENABLE_PREPROCESSING']:
    import tempfile
    from common.image_preprocessing import (
        enhance_statement_quality,
        enhance_for_llama,
        preprocess_statement_for_llama,
        adaptive_enhance,
        preprocess_recommended
    )
    
    preprocess_functions = {
        'light': enhance_statement_quality,
        'moderate': enhance_for_llama,
        'aggressive': preprocess_statement_for_llama,
        'adaptive': adaptive_enhance,
        'recommended': preprocess_recommended
    }
    
    preprocess_fn = preprocess_functions[CONFIG['PREPROCESSING_MODE']]
    preprocessed_images = []
    
    rprint(f"[cyan]🔧 Preprocessing {len(all_images)} images (mode: {CONFIG['PREPROCESSING_MODE']})[/cyan]")
    
    if CONFIG['SAVE_PREPROCESSED']:
        preprocessed_dir = Path(CONFIG['PREPROCESSED_DIR'] or 'preprocessed_images')
        preprocessed_dir.mkdir(parents=True, exist_ok=True)
    else:
        preprocessed_dir = Path(tempfile.mkdtemp(prefix='preprocessed_'))
    
    for img_path in all_images:
        original_filename = Path(img_path).name
        try:
            preprocessed_img = preprocess_fn(img_path)
            preprocessed_path = preprocessed_dir / original_filename
            preprocessed_img.save(preprocessed_path)
            preprocessed_images.append(str(preprocessed_path))
        except Exception as e:
            rprint(f"[yellow]⚠️  Preprocessing failed for {original_filename}: {e}[/yellow]")
            preprocessed_images.append(img_path)
    
    all_images = preprocessed_images
    rprint(f"[green]✅ Preprocessing complete[/green]")

# Load ground truth
ground_truth_path = Path(CONFIG['GROUND_TRUTH'])
if not ground_truth_path.is_absolute():
    ground_truth_path = Path.cwd() / ground_truth_path

ground_truth = load_ground_truth(str(ground_truth_path), verbose=CONFIG['VERBOSE'])

# Apply max images limit
if CONFIG['MAX_IMAGES']:
    all_images = all_images[:CONFIG['MAX_IMAGES']]

rprint(f"[bold green]Ready to process {len(all_images)} images[/bold green]")
rprint(f"[cyan]Ground truth loaded for {len(ground_truth)} images[/cyan]")

In [ ]:
#Cell 7.75
# Ground Truth Debug - validate GT matching before processing
if ground_truth:
    rprint("\n[bold yellow]🔍 Ground Truth Debug Info[/bold yellow]")
    rprint(f"[cyan]Total ground truth entries: {len(ground_truth)}[/cyan]")
    rprint(f"[cyan]Total images to process: {len(all_images)}[/cyan]")
    
    # Show first 3 ground truth keys
    gt_keys = list(ground_truth.keys())[:3]
    rprint(f"[cyan]Sample GT keys: {gt_keys}[/cyan]")
    
    # Show first 3 image filenames (with and without extensions)
    img_names_full = [Path(img).name for img in all_images[:3]]
    img_names_no_ext = [Path(img).stem for img in all_images[:3]]
    rprint(f"[cyan]Sample image names (full): {img_names_full}[/cyan]")
    rprint(f"[cyan]Sample image names (no ext): {img_names_no_ext}[/cyan]")
    
    # Check for mismatches using filename WITHOUT extension (Path.stem)
    missing_gt = []
    for img in all_images:
        img_name_no_ext = Path(img).stem  # Strip extension for GT lookup
        if img_name_no_ext not in ground_truth:
            missing_gt.append(Path(img).name)  # Show full name in error
    
    if missing_gt:
        rprint(f"[red]⚠️  WARNING: {len(missing_gt)} images missing from ground truth![/red]")
        rprint(f"[red]First 5 missing: {missing_gt[:5]}[/red]")
    else:
        rprint(f"[green]✅ All {len(all_images)} images have ground truth entries (using stem lookup)[/green]")
    
    console.rule()
else:
    rprint("[yellow]⚠️  No ground truth loaded (inference-only mode)[/yellow]")

In [ ]:
#Cell 8
# Load document-specific field mappings for evaluation
field_defs_path = Path('config/field_definitions.yaml')
with open(field_defs_path, 'r') as f:
    field_defs = yaml.safe_load(f)

DOC_TYPE_FIELDS = {
    'invoice': field_defs['document_fields']['invoice']['fields'],
    'receipt': field_defs['document_fields']['receipt']['fields'],
    'bank_statement': field_defs['document_fields']['bank_statement']['fields'],
    'statement': field_defs['document_fields']['bank_statement']['fields'],  # Alias
}

# Oracle batch processing - use ground truth doc type to select prompt
console.rule("[bold cyan]Oracle Batch Processing[/bold cyan]")

batch_results = []
processing_times = []
skipped_count = 0

for idx, image_path in enumerate(all_images, 1):
    # CRITICAL FIX: Use stem (no extension) to match ground truth keys
    # Ground truth image_name column has no file extensions
    image_name = Path(image_path).stem
    image_display_name = Path(image_path).name  # For display only
    
    rprint(f"\n[bold cyan]Processing {idx}/{len(all_images)}: {image_display_name}[/bold cyan]")
    
    # Check if ground truth exists (using name without extension)
    if image_name not in ground_truth:
        rprint(f"[yellow]⚠️  No ground truth for {image_name} - skipping[/yellow]")
        skipped_count += 1
        continue
    
    gt_data = ground_truth[image_name]
    gt_doc_type = gt_data.get('DOCUMENT_TYPE', '').lower()
    
    # Map ground truth doc type to prompt key
    if 'statement' in gt_doc_type or 'bank' in gt_doc_type:
        prompt_key = 'bank_statement'
    elif 'invoice' in gt_doc_type:
        prompt_key = 'invoice'
    elif 'receipt' in gt_doc_type:
        prompt_key = 'receipt'
    else:
        rprint(f"[red]❌ Unknown doc type: {gt_doc_type} - skipping[/red]")
        skipped_count += 1
        continue
    
    # Get oracle prompt
    if prompt_key not in ORACLE_PROMPTS:
        rprint(f"[red]❌ No prompt for {prompt_key} - skipping[/red]")
        skipped_count += 1
        continue
    
    oracle_prompt = ORACLE_PROMPTS[prompt_key]
    
    rprint(f"[cyan]📋 Using oracle prompt: {prompt_key} (from ground truth: {gt_doc_type})[/cyan]")
    
    # Load and process image
    try:
        image = PILImage.open(image_path)
        
        # Create messages in Llama chat format
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": oracle_prompt}
                ]
            }
        ]
        
        # Apply chat template
        input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
        
        # Process inputs
        inputs = processor(
            image,
            input_text,
            return_tensors="pt"
        ).to(model.device)
        
        # Generate response
        start_time = datetime.now()
        
        with torch.inference_mode():
            output = model.generate(
                **inputs,
                max_new_tokens=CONFIG['MAX_NEW_TOKENS'],
                do_sample=False
            )
        
        processing_time = (datetime.now() - start_time).total_seconds()
        processing_times.append(processing_time)
        
        # Decode response
        response = processor.decode(output[0], skip_special_tokens=True)
        
        # Extract only the assistant's response (after "assistant\n\n")
        if "assistant\n\n" in response:
            response = response.split("assistant\n\n", 1)[1]
        
        # Parse extraction response (field-by-field format, NOT JSON)
        extracted_data = parse_extraction_response(response)
        
        # Manual evaluation - get relevant fields for this doc type
        doc_type_normalized = gt_doc_type.replace(' ', '_')
        relevant_fields = DOC_TYPE_FIELDS.get(doc_type_normalized, DOC_TYPE_FIELDS.get(prompt_key, []))
        
        # Evaluate each field
        field_scores = {}
        total_f1 = 0.0
        fields_evaluated = 0
        fields_matched = 0
        
        for field in relevant_fields:
            extracted_value = extracted_data.get(field, "NOT_FOUND")
            gt_value = gt_data.get(field, "NOT_FOUND")
            
            # Skip if both are NOT_FOUND
            if extracted_value == "NOT_FOUND" and gt_value == "NOT_FOUND":
                continue
            
            fields_evaluated += 1
            
            try:
                metrics = calculate_field_accuracy_with_method(
                    extracted_value, gt_value, field, 
                    method=os.environ.get('EVALUATION_METHOD', 'order_aware_f1')
                )
            except Exception as e:
                rprint(f"[yellow]⚠️  Error evaluating {field}: {e}[/yellow]")
                metrics = {'f1_score': 0.0, 'precision': 0.0, 'recall': 0.0}
            
            field_scores[field] = metrics
            total_f1 += metrics.get('f1_score', 0.0)
            
            if metrics.get('f1_score', 0.0) > 0.9:
                fields_matched += 1
        
        # Calculate overall accuracy
        overall_accuracy = (total_f1 / fields_evaluated) if fields_evaluated > 0 else 0.0
        
        # Store evaluation results
        evaluation = {
            'overall_accuracy': overall_accuracy,
            'fields_evaluated': fields_evaluated,
            'fields_matched': fields_matched,
            'total_fields': len(relevant_fields),
            'field_scores': field_scores
        }
        
        # Store results (use stem for image_name to match GT)
        result = {
            'image_name': image_name,  # Without extension - matches GT
            'image_path': image_path,
            'oracle_doc_type': gt_doc_type,
            'prompt_used': prompt_key,
            'extracted_data': extracted_data,
            'evaluation': evaluation,
            'processing_time': processing_time,
            'raw_response': response
        }
        
        batch_results.append(result)
        
        # Show summary
        accuracy = evaluation.get('overall_accuracy', 0) * 100
        fields_matched = evaluation.get('fields_matched', 0)
        total_fields = evaluation.get('total_fields', 0)
        
        rprint(f"[green]✅ Accuracy: {accuracy:.1f}% ({fields_matched}/{total_fields} fields)[/green]")
        rprint(f"[cyan]⏱️  Time: {processing_time:.2f}s[/cyan]")
        
    except Exception as e:
        rprint(f"[red]❌ Error processing {image_display_name}: {e}[/red]")
        import traceback
        rprint(f"[red]{traceback.format_exc()}[/red]")
        continue

console.rule("[bold green]Oracle Processing Complete[/bold green]")
rprint(f"[green]✅ Processed: {len(batch_results)} images[/green]")
rprint(f"[yellow]⚠️  Skipped: {skipped_count} images[/yellow]")

if processing_times:
    avg_time = np.mean(processing_times)
    avg_accuracy = np.mean([r['evaluation']['overall_accuracy'] * 100 for r in batch_results])
    rprint(f"[cyan]Average time: {avg_time:.2f}s[/cyan]")
    rprint(f"[cyan]Average accuracy: {avg_accuracy:.1f}%[/cyan]")

In [ ]:
#Cell 8.5
# Load field columns for CSV export
field_defs_path = Path('config/field_definitions.yaml')

with open(field_defs_path, 'r') as f:
    field_defs = yaml.safe_load(f)

# Get universal fields from YAML (19 total)
all_universal_fields = field_defs['document_fields']['universal']['fields']

# Remove fields no longer extracted (TRANSACTION_AMOUNTS_RECEIVED, ACCOUNT_BALANCE)
# Final 17 fields for CSV columns
EXCLUDED_FIELDS = ['TRANSACTION_AMOUNTS_RECEIVED', 'ACCOUNT_BALANCE']
FIELD_COLUMNS = [f for f in all_universal_fields if f not in EXCLUDED_FIELDS]

rprint(f"[green]✅ Loaded {len(FIELD_COLUMNS)} field columns for CSV export[/green]")
rprint(f"[cyan]Excluded: {', '.join(EXCLUDED_FIELDS)}[/cyan]")

In [ ]:
#Cell 9
# Create oracle CSV results file
# Match structure expected by model_comparison.ipynb
# FIELD_COLUMNS loaded from YAML in previous cell

oracle_csv_data = []

for i, result in enumerate(batch_results):
    image_name = result['image_name']
    oracle_doc_type = result['oracle_doc_type']
    prompt_used = f"llama_oracle_{result['prompt_used']}"
    processing_time = result['processing_time']
    
    extracted_data = result['extracted_data']
    evaluation = result['evaluation']
    
    # Count fields
    found_fields = sum(1 for field in FIELD_COLUMNS if extracted_data.get(field, 'NOT_FOUND') != 'NOT_FOUND')
    field_coverage = (found_fields / len(FIELD_COLUMNS) * 100) if FIELD_COLUMNS else 0
    
    # Create row
    row_data = {
        'image_file': image_name,
        'image_name': image_name,
        'document_type': oracle_doc_type,
        'oracle_doc_type': oracle_doc_type,
        'processing_time': processing_time,
        'field_count': evaluation.get('total_fields', 0),
        'found_fields': evaluation.get('fields_extracted', 0),
        'field_coverage': field_coverage,
        'prompt_used': prompt_used,
        'timestamp': datetime.now().isoformat(),
        'overall_accuracy': evaluation.get('overall_accuracy', 0) * 100,
        'fields_extracted': evaluation.get('fields_extracted', 0),
        'fields_matched': evaluation.get('fields_matched', 0),
        'total_fields': evaluation.get('total_fields', 0)
    }
    
    # Add all field values
    for field in FIELD_COLUMNS:
        row_data[field] = extracted_data.get(field, 'NOT_FOUND')
    
    oracle_csv_data.append(row_data)

# Create DataFrame and save
oracle_df = pd.DataFrame(oracle_csv_data)
oracle_csv_path = OUTPUT_DIRS['csv'] / f"llama_oracle_batch_results_{BATCH_TIMESTAMP}.csv"
oracle_df.to_csv(oracle_csv_path, index=False)

rprint("[bold green]✅ Oracle CSV exported:[/bold green]")
rprint(f"[cyan]📄 File: {oracle_csv_path}[/cyan]")
rprint(f"[cyan]📊 Structure: {len(oracle_df)} rows × {len(oracle_df.columns)} columns[/cyan]")
rprint("[cyan]🔗 Compatible with model_comparison.ipynb pattern: *llama*oracle*batch*results*.csv[/cyan]")

# Display sample
rprint("\n[bold blue]📋 Sample exported data:[/bold blue]")
sample_cols = ['image_file', 'oracle_doc_type', 'overall_accuracy', 'processing_time', 'fields_matched', 'total_fields']
if len(oracle_df) > 0:
    display(oracle_df[sample_cols].head(3))
else:
    rprint("[yellow]⚠️ No data to display[/yellow]")

In [ ]:
#Cell 10
# Summary statistics by document type
console.rule("[bold cyan]Oracle Results by Document Type[/bold cyan]")

# Group by oracle doc type
doc_type_stats = {}

for result in batch_results:
    doc_type = result['oracle_doc_type']
    
    if doc_type not in doc_type_stats:
        doc_type_stats[doc_type] = {
            'count': 0,
            'accuracies': [],
            'times': [],
            'fields_matched': [],
            'total_fields': []
        }
    
    stats = doc_type_stats[doc_type]
    evaluation = result['evaluation']
    
    stats['count'] += 1
    stats['accuracies'].append(evaluation.get('overall_accuracy', 0) * 100)
    stats['times'].append(result['processing_time'])
    stats['fields_matched'].append(evaluation.get('fields_matched', 0))
    stats['total_fields'].append(evaluation.get('total_fields', 0))

# Display statistics
for doc_type, stats in doc_type_stats.items():
    console.rule(f"[cyan]{doc_type.upper()}[/cyan]")
    
    rprint(f"[cyan]Count: {stats['count']}[/cyan]")
    rprint(f"[cyan]Average accuracy: {np.mean(stats['accuracies']):.1f}%[/cyan]")
    rprint(f"[cyan]Average time: {np.mean(stats['times']):.2f}s[/cyan]")
    rprint(f"[cyan]Fields matched: {np.mean(stats['fields_matched']):.1f}/{np.mean(stats['total_fields']):.1f}[/cyan]")

In [ ]:
#Cell 11
# Comparison: Oracle vs Expected Performance
console.rule("[bold cyan]Oracle Performance Analysis[/bold cyan]")

rprint("[bold]Key Insights:[/bold]")
rprint("\n[cyan]1. Perfect Classification:[/cyan]")
rprint("   Oracle method uses ground truth doc type - no classification errors")
rprint("   Always extracts correct field set for each document type")

rprint("\n[cyan]2. Performance Metrics:[/cyan]")
if batch_results:
    avg_oracle_accuracy = np.mean([r['evaluation']['overall_accuracy'] * 100 for r in batch_results])
    rprint(f"   Oracle accuracy: {avg_oracle_accuracy:.1f}%")
    rprint("   This represents extraction accuracy WITHOUT classification penalty")

rprint("\n[cyan]3. Comparison Analysis:[/cyan]")
rprint("   Compare this CSV with baseline (llama_batch_results_*.csv) to quantify:")
rprint("   • Classification penalty = Baseline accuracy - Oracle accuracy")
rprint("   • Extraction ceiling = Maximum achievable with perfect classification")

rprint(f"\n[bold green]✅ Oracle results saved to: {oracle_csv_path}[/bold green]")